In [ ]:
!pip install -U --pre tensorflow=="2.*"
!pip install tf_slim

In [ ]:
!pip install pycocotools

In [8]:
import os
import pathlib


if "models" in pathlib.Path.cwd().parts:
    while "models" in pathlib.Path.cwd().parts:
        os.chdir('..')
elif not pathlib.Path('models').exists():
    !git clone --depth 1 https://github.com/tensorflow/models

In [1]:
%%cmd #veya %%bash
cd models/research/
protoc object_detection/protos/*.proto --python_out=.

Microsoft Windows [Version 10.0.19041.685]
(c) 2020 Microsoft Corporation. Tüm hakları saklıdır.

(base) C:\Users\burak\SheepObjectDetect>cd models/research/

(base) C:\Users\burak\SheepObjectDetect\models\research>protoc object_detection/protos/*.proto --python_out=.

(base) C:\Users\burak\SheepObjectDetect\models\research>

In [4]:
%%cmd 
cd models/research
pip install .

Microsoft Windows [Version 10.0.19041.685]
(c) 2020 Microsoft Corporation. Tüm hakları saklıdır.

(base) C:\Users\burak\SheepObjectDetect>cd models/research

(base) C:\Users\burak\SheepObjectDetect\models\research>pip install .
Processing c:\users\burak\sheepobjectdetect\models\research
  Created wheel for object-detection: filename=object_detection-0.1-py3-none-any.whl size=1387470 sha256=460960091381099f69c05618956e4e598e17d2502a8fbff85a33e618a63fe3ae
  Stored in directory: C:\Users\burak\AppData\Local\Temp\pip-ephem-wheel-cache-5y6q1xqi\wheels\6b\af\3f\80e41baabe19e7f017240d8c3ce7d93a7e47a6d1b04fcf3a65
Successfully built object-detection
  Attempting uninstall: object-detection
    Found existing installation: object-detection 0.1
    Uninstalling object-detection-0.1:
      Successfully uninstalled object-detection-0.1

(base) C:\Users\burak\SheepObjectDetect\models\research>

### Imports

In [2]:
import numpy as np
import os
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile

from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image
from IPython.display import display

In [3]:
from object_detection.utils import ops as utils_ops
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as vis_util

In [4]:
utils_ops.tf = tf.compat.v1

tf.gfile = tf.io.gfile

In [5]:
def load_model(model_name):
    base_url = 'http://download.tensorflow.org/models/object_detection/'
    model_file = model_name + '.tar.gz'
    model_dir = tf.keras.utils.get_file(
        fname=model_name, 
        origin=base_url + model_file,
        untar=True)

    model_dir = pathlib.Path(model_dir)/"saved_model"

    model = tf.saved_model.load(str(model_dir))
    model = model.signatures['serving_default']

    return model

In [6]:
PATH_TO_LABELS = 'models/research/object_detection/data/mscoco_label_map.pbtxt'
category_index = label_map_util.create_category_index_from_labelmap(PATH_TO_LABELS, use_display_name=True)

In [9]:
model_name = 'ssd_mobilenet_v1_coco_2017_11_17'
detection_model = load_model(model_name)

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


In [10]:
print(detection_model.inputs)

[<tf.Tensor 'image_tensor:0' shape=(None, None, None, 3) dtype=uint8>]


In [11]:
detection_model.output_dtypes

{'detection_boxes': tf.float32,
 'detection_scores': tf.float32,
 'detection_classes': tf.float32,
 'num_detections': tf.float32}

In [12]:
detection_model.output_shapes

{'detection_boxes': TensorShape([None, 100, 4]),
 'detection_scores': TensorShape([None, 100]),
 'detection_classes': TensorShape([None, 100]),
 'num_detections': TensorShape([None])}

In [13]:
def run_inference_for_single_image(model, image):
    image = np.asarray(image)
    input_tensor = tf.convert_to_tensor(image)
    input_tensor = input_tensor[tf.newaxis,...]
    
    output_dict = model(input_tensor)

    num_detections = int(output_dict.pop('num_detections'))
    output_dict = {key:value[0, :num_detections].numpy() 
                   for key,value in output_dict.items()}
    output_dict['num_detections'] = num_detections

    output_dict['detection_classes'] = output_dict['detection_classes'].astype(np.int64)
   
    if 'detection_masks' in output_dict:
        detection_masks_reframed = utils_ops.reframe_box_masks_to_image_masks(
                                    output_dict['detection_masks'], output_dict['detection_boxes'],
                                    image.shape[0], image.shape[1])      
        detection_masks_reframed = tf.cast(detection_masks_reframed > 0.5, tf.uint8)
        output_dict['detection_masks_reframed'] = detection_masks_reframed.numpy()
    
    return output_dict

In [14]:
import cv2
cap = cv2.VideoCapture('C:/Users/burak/SheepObjectDetect/koyunvideo1.mp4')

def run_inference(model, cap):
    while cap.isOpened():
        ret, image_np = cap.read()
        output_dict = run_inference_for_single_image(model, image_np)
        vis_util.visualize_boxes_and_labels_on_image_array(
            image_np,
            output_dict['detection_boxes'],
            output_dict['detection_classes'],
            output_dict['detection_scores'],
            category_index,
            instance_masks=output_dict.get('detection_masks_reframed', None),
            use_normalized_coordinates=True,
            line_thickness=8)
        cv2.imshow('object_detection', cv2.resize(image_np, (800, 600)))
        if cv2.waitKey(25) & 0xFF == ord('q'):
            cap.release()
            cv2.destroyAllWindows()
            break

run_inference(detection_model, cap)